In [1]:
# Required Libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
# Load datasets
sector_inter = pd.read_csv('sector2_inter.csv', parse_dates=['RowKey']).set_index('RowKey')
future_data = pd.read_csv('future_data2.csv', parse_dates=['RowKey']).set_index('RowKey')

In [3]:
# Splitting the data
train, valid = train_test_split(sector_inter, test_size=0.2, shuffle=False)
test = future_data

In [4]:
# Drop INF_Value for XGBoost training and validation
X_train = train.drop('INF_Value', axis=1)
y_train = train['INF_Value']
X_valid = valid.drop('INF_Value', axis=1)
y_valid = valid['INF_Value']

In [5]:
# SARIMA Model
sarima_model = SARIMAX(y_train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sarima_result = sarima_model.fit(disp=False)
sarima_predictions = sarima_result.predict(start=y_train.index[-1], end=y_valid.index[-1], dynamic=True)
future_sarima_predictions = sarima_result.predict(start=y_valid.index[-1], end=test.index[-1], dynamic=True)

C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\statespace\kalman_filter.py:2473: ValueWarning: Dynamic prediction specified to begin during out-of-sample forecasting period, and so has no effect.
  warn('Dynamic prediction specified to begin during'


In [6]:
test

,INF_Value,precip_lag1,precip_lag2,precip_lag3,precip_lag4,precip_lag5,precip_lag6,precip_lag7,precip_rolling_sum,precip_rolling_mean,...,temp_lag1,temp_lag2,temp_lag3,humidity_lag1,humidity_lag2,humidity_lag3,temp_rolling_mean,temp_rolling_std,humidity_rolling_mean,humidity_rolling_std
RowKey,,,,,,,,,,,,,,,,,,,,,
2022-12-13,NaN,3.4,31.6,0.6,7.0,11.8,0.0,0.6,76.2,10.885714,...,14.5,12.0,10.5,86.12,91.03,87.26,14.166667,2.020726,89.000000,2.562987
2022-12-14,NaN,21.8,3.4,31.6,0.6,7.0,11.8,0.0,77.2,11.028571,...,16.0,14.5,12.0,89.85,86.12,91.03,14.633333,1.305118,87.836667,1.882613
2022-12-15,NaN,1.0,21.8,3.4,31.6,0.6,7.0,11.8,65.8,9.400000,...,13.4,16.0,14.5,87.54,89.85,86.12,13.300000,2.751363,87.266667,2.730281
2022-12-16,NaN,0.4,1.0,21.8,3.4,31.6,0.6,7.0,63.4,9.057143,...,10.5,13.4,16.0,84.41,87.54,89.85,11.000000,2.193171,88.370000,4.433655
2022-12-17,NaN,4.6,0.4,1.0,21.8,3.4,31.6,0.6,66.4,9.485714,...,9.1,10.5,13.4,93.16,84.41,87.54,9.700000,0.721110,90.390000,5.183503
2022-12-18,NaN,3.6,4.6,0.4,1.0,21.8,3.4,31.6,34.8,4.971429,...,9.5,9.1,10.5,93.60,93.16,84.41,8.200000,1.915724,94.123333,1.306152
2022-12-19,NaN,0.0,3.6,4.6,0.4,1.0,21.8,3.4,31.4,4.485714,...,6.0,9.5,9.1,95.61,93.60,93.16,6.766667,2.441994,95.346667,1.631022


In [7]:
X_test = test.drop('INF_Value', axis=1)

In [8]:
# XGBoost Model
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)
# XGBoost predictions
xgb_predictions = xgb_model.predict(X_valid)
future_xgb_predictions = xgb_model.predict(X_test)

In [9]:
# Convert xgb_predictions to a pandas Series with a datetime index
xgb_predictions_series = pd.Series(xgb_predictions, index=y_valid.index)

# Ensemble Predictions
ensemble_predictions = 0.5 * sarima_predictions + 0.5 * xgb_predictions_series


In [10]:
future_xgb_predictions_series = pd.Series(future_xgb_predictions, index=test.index)
future_ensemble_predictions = 0.5 * future_sarima_predictions + 0.5 * future_xgb_predictions_series


In [11]:
# Create DataFrame for Test and Future Predictions
predictions_df = pd.DataFrame({
    'Actual': y_valid,
    'SARIMA_Predictions': sarima_predictions,
    'XGBoost_Predictions': xgb_predictions_series,
    'Ensemble_Predictions': ensemble_predictions
}, index=y_valid.index)

future_predictions_df = pd.DataFrame({
    'SARIMA_Future_Predictions': future_sarima_predictions,
    'XGBoost_Future_Predictions': future_xgb_predictions_series,
    'Ensemble_Future_Predictions': future_ensemble_predictions
}, index=test.index)


In [12]:
future_predictions_df

,SARIMA_Future_Predictions,XGBoost_Future_Predictions,Ensemble_Future_Predictions
RowKey,,,
2022-12-13,1160.310060,768.658936,964.484498
2022-12-14,1167.700412,766.487244,967.093828
2022-12-15,1174.511150,782.446350,978.478750
2022-12-16,1170.123532,768.981506,969.552519
2022-12-17,1160.139392,754.453491,957.296442
2022-12-18,1153.669030,776.475159,965.072094
2022-12-19,1177.824382,746.041565,961.932974


In [13]:
predictions_df

,Actual,SARIMA_Predictions,XGBoost_Predictions,Ensemble_Predictions
RowKey,,,,
2022-09-04,1267.1,1224.593374,1261.640015,1243.116694
2022-09-05,1201.8,1216.437112,1008.461548,1112.449330
2022-09-06,1199.3,1211.525367,1010.097046,1110.811206
2022-09-07,1237.2,1189.719859,1007.687439,1098.703649
2022-09-08,1166.8,1177.548059,1243.731323,1210.639691
...,...,...,...,...
2022-12-08,1028.9,1185.962876,797.645874,991.804375
2022-12-09,1046.6,1194.398716,757.417969,975.908342
2022-12-10,1053.5,1187.607967,754.374023,970.990995


In [14]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Actual values
actual_values = predictions_df['Actual']

# SARIMA predictions
sarima_predictions = predictions_df['SARIMA_Predictions']

# XGBoost predictions
xgboost_predictions = predictions_df['XGBoost_Predictions']

# Ensemble predictions
ensemble_predictions = predictions_df['Ensemble_Predictions']

# Calculate MAPE for each method
def mape(y_true, y_pred):
    return 100 * np.mean(np.abs((y_true - y_pred) / y_true))

mape_sarima = mape(actual_values, sarima_predictions)
mape_xgboost = mape(actual_values, xgboost_predictions)
mape_ensemble = mape(actual_values, ensemble_predictions)

# Calculate RMSE for each method
rmse_sarima = np.sqrt(mean_squared_error(actual_values, sarima_predictions))
rmse_xgboost = np.sqrt(mean_squared_error(actual_values, xgboost_predictions))
rmse_ensemble = np.sqrt(mean_squared_error(actual_values, ensemble_predictions))

# Calculate MAE for each method
mae_sarima = mean_absolute_error(actual_values, sarima_predictions)
mae_xgboost = mean_absolute_error(actual_values, xgboost_predictions)
mae_ensemble = mean_absolute_error(actual_values, ensemble_predictions)

# Display the results
print("SARIMA MAPE:", mape_sarima)
print("XGBoost MAPE:", mape_xgboost)
print("Ensemble MAPE:", mape_ensemble)

print("SARIMA RMSE:", rmse_sarima)
print("XGBoost RMSE:", rmse_xgboost)
print("Ensemble RMSE:", rmse_ensemble)

print("SARIMA MAE:", mae_sarima)
print("XGBoost MAE:", mae_xgboost)
print("Ensemble MAE:", mae_ensemble)


SARIMA MAPE: 38.50563288298273
XGBoost MAPE: 41.0682247218467
Ensemble MAPE: 35.35074160275359
SARIMA RMSE: 268.0212712757236
XGBoost RMSE: 342.4299947067038
Ensemble RMSE: 251.29899373459023
SARIMA MAE: 163.22305148118946
XGBoost MAE: 309.7257781982422
Ensemble MAE: 189.13789705344738


In [15]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6],
    'min_child_weight': [1, 2, 3],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'n_estimators': [50, 100, 200]
}

# Create the XGBoost model
xgb_model = XGBRegressor()

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to your training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train a new XGBoost model with the best hyperparameters
best_xgb_model = XGBRegressor(**best_params)
best_xgb_model.fit(X_train, y_train)

# XGBoost predictions with the best hyperparameters
xgb_predictions = best_xgb_model.predict(X_valid)
future_xgb_predictions = best_xgb_model.predict(X_test)


Best Hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.7}


In [16]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools

# Define the range of orders and seasonal orders to search through
p_values = [0, 1, 2]  # AutoRegressive order (p)
d_values = [0, 1]     # Differencing order (d)
q_values = [0, 1, 2]  # Moving Average order (q)
seasonal_periods = [7, 30, 13, 12]  # Different seasonal periods to try

best_aic = float("inf")
best_order = (0, 0, 0)
best_seasonal_order = (0, 0, 0, 0)

# Iterate through all possible combinations
for p in p_values:
    for d in d_values:
        for q in q_values:
            for seasonal_period in seasonal_periods:
                order = (p, d, q)
                seasonal_order = (p, d, q, seasonal_period)
                
                # Fit the SARIMA model with the current order and seasonal order
                sarima_model = SARIMAX(y_train, order=order, seasonal_order=seasonal_order)
                sarima_result = sarima_model.fit(disp=False)
                
                # Calculate AIC (Akaike Information Criterion)
                current_aic = sarima_result.aic
                
                # Update the best order if the current AIC is lower
                if current_aic < best_aic:
                    best_aic = current_aic
                    best_order = order
                    best_seasonal_order = seasonal_order

# Print the best SARIMA order and seasonal order
print("Best SARIMA Order:", best_order)
print("Best Seasonal Order:", best_seasonal_order)

# Fit the SARIMA model with the best order and seasonal order
best_sarima_model = SARIMAX(y_train, order=best_order, seasonal_order=best_seasonal_order)
best_sarima_result = best_sarima_model.fit(disp=False)

# SARIMA predictions with the best orders
sarima_predictions = best_sarima_result.predict(start=y_train.index[-1], end=y_valid.index[-1], dynamic=True)
future_sarima_predictions = best_sarima_result.predict(start=y_valid.index[-1], end=test.index[-1], dynamic=True)


C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCac

Best SARIMA Order: (0, 1, 1)
Best Seasonal Order: (0, 1, 1, 30)


C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\lalit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\statespace\kalman_filter.py:2473: ValueWarning: Dynamic prediction specified to begin during out-of-sample forecasting period, and so has no effect.
  warn('Dynamic prediction specified to begin during'


In [17]:
# Convert xgb_predictions to a pandas Series with a datetime index
xgb_predictions_series = pd.Series(xgb_predictions, index=y_valid.index)



In [18]:
future_xgb_predictions_series = pd.Series(future_xgb_predictions, index=test.index)
future_ensemble_predictions = 0.5 * future_sarima_predictions + 0.5 * future_xgb_predictions_series


In [36]:
future_ensemble_predictions = 0.75 * future_sarima_predictions + 0.25 * future_xgb_predictions_series

In [37]:
# Ensemble Predictions
ensemble_predictions = 0.75 * sarima_predictions + 0.25 * xgb_predictions_series

In [38]:
# Create DataFrame for Test and Future Predictions
predictions_df = pd.DataFrame({
    'Actual': y_valid,
    'SARIMA_Predictions': sarima_predictions,
    'XGBoost_Predictions': xgb_predictions_series,
    'Ensemble_Predictions': ensemble_predictions
}, index=y_valid.index)

future_predictions_df = pd.DataFrame({
    'SARIMA_Future_Predictions': future_sarima_predictions,
    'XGBoost_Future_Predictions': future_xgb_predictions_series,
    'Ensemble_Future_Predictions': future_ensemble_predictions
}, index=test.index)

In [39]:
predictions_df

,Actual,SARIMA_Predictions,XGBoost_Predictions,Ensemble_Predictions
RowKey,,,,
2022-09-04,1267.1,1230.289506,1034.798218,1181.416684
2022-09-05,1201.8,1214.602106,855.056274,1124.715648
2022-09-06,1199.3,1213.009073,856.341553,1123.842193
2022-09-07,1237.2,1226.576075,815.696167,1123.856098
2022-09-08,1166.8,1243.474997,987.177185,1179.400544
...,...,...,...,...
2022-12-08,1028.9,1261.530237,805.033752,1147.406116
2022-12-09,1046.6,1251.354495,801.785034,1138.962130
2022-12-10,1053.5,1256.493203,801.785034,1142.816161


In [34]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Actual values
actual_values = predictions_df['Actual']

# SARIMA predictions
sarima_predictions = predictions_df['SARIMA_Predictions']

# XGBoost predictions
xgboost_predictions = predictions_df['XGBoost_Predictions']

# Ensemble predictions
ensemble_predictions = predictions_df['Ensemble_Predictions']

# Calculate MAPE for each method
def mape(y_true, y_pred):
    return 100 * np.mean(np.abs((y_true - y_pred) / y_true))

mape_sarima = mape(actual_values, sarima_predictions)
mape_xgboost = mape(actual_values, xgboost_predictions)
mape_ensemble = mape(actual_values, ensemble_predictions)

# Calculate RMSE for each method
rmse_sarima = np.sqrt(mean_squared_error(actual_values, sarima_predictions))
rmse_xgboost = np.sqrt(mean_squared_error(actual_values, xgboost_predictions))
rmse_ensemble = np.sqrt(mean_squared_error(actual_values, ensemble_predictions))

# Calculate MAE for each method
mae_sarima = mean_absolute_error(actual_values, sarima_predictions)
mae_xgboost = mean_absolute_error(actual_values, xgboost_predictions)
mae_ensemble = mean_absolute_error(actual_values, ensemble_predictions)

# Display the results
print("SARIMA MAPE:", mape_sarima)
print("XGBoost MAPE:", mape_xgboost)
print("Ensemble MAPE:", mape_ensemble)

print("SARIMA RMSE:", rmse_sarima)
print("XGBoost RMSE:", rmse_xgboost)
print("Ensemble RMSE:", rmse_ensemble)

print("SARIMA MAE:", mae_sarima)
print("XGBoost MAE:", mae_xgboost)
print("Ensemble MAE:", mae_ensemble)


SARIMA MAPE: 42.51793438829707
XGBoost MAPE: 41.68864320052476
Ensemble MAPE: 35.17826233813729
SARIMA RMSE: 288.39845676162633
XGBoost RMSE: 337.87139367534417
Ensemble RMSE: 250.1507121191479
SARIMA MAE: 198.72626524573636
XGBoost MAE: 314.38147888183585
Ensemble MAE: 149.21390583584142


In [31]:
predictions_df

,Actual,SARIMA_Predictions,XGBoost_Predictions,Ensemble_Predictions
RowKey,,,,
2022-09-04,1267.1,1230.289506,1034.798218,1191.191246
2022-09-05,1201.8,1214.602106,855.056274,1142.692945
2022-09-06,1199.3,1213.009073,856.341553,1141.675569
2022-09-07,1237.2,1226.576075,815.696167,1144.400096
2022-09-08,1166.8,1243.474997,987.177185,1192.215438
...,...,...,...,...
2022-12-08,1028.9,1261.530237,805.033752,1170.230949
2022-12-09,1046.6,1251.354495,801.785034,1161.440606
2022-12-10,1053.5,1256.493203,801.785034,1165.551573


In [35]:
#actual mean
actual_mean = predictions_df['Actual'].mean()
actual_mean

1074.3670000000002

In [44]:
future_predictions_df['Ensemble_Future_Predictions']

RowKey
2022-12-13    1165.146266
2022-12-14    1159.185854
2022-12-15    1160.388389
2022-12-16    1160.159465
2022-12-17    1138.886077
2022-12-18    1096.057928
2022-12-19    1093.107747
Name: Ensemble_Future_Predictions, dtype: float64